In [1]:
import pandas as pd
import numpy as np

cars = pd.read_csv("imports-85.data")

#create first ligne
colName = ["symboling","normalized-losses","make","fuel-type","aspiration","num-of-doors","body-style","drive-wheels","engine-location","wheel-base","length","width","height","curb-weight","engine-type","num-of-cylinders","engine-size","fuel-system","bore","stroke","compression-ratio","horsepower","peak-rpm","city-mpg","highway-mpg","price"]
cars.columns = colName

#info of dataframe
#cars.info()

total_rows = len(cars.index)
print(total_rows)

#cars.head(20)

cars.info()



204
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          204 non-null    int64  
 1   normalized-losses  204 non-null    object 
 2   make               204 non-null    object 
 3   fuel-type          204 non-null    object 
 4   aspiration         204 non-null    object 
 5   num-of-doors       204 non-null    object 
 6   body-style         204 non-null    object 
 7   drive-wheels       204 non-null    object 
 8   engine-location    204 non-null    object 
 9   wheel-base         204 non-null    float64
 10  length             204 non-null    float64
 11  width              204 non-null    float64
 12  height             204 non-null    float64
 13  curb-weight        204 non-null    int64  
 14  engine-type        204 non-null    object 
 15  num-of-cylinders   204 non-null    object 
 16  engine-size        204

### Feature engineering : processus de traitement et de création de nouvelles caractéristiques

In [2]:
#from documentation, get continuous data
cols_continuous = ['normalized-losses','wheel-base','length','width','height','curb-weight','engine-size','bore','stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg','price']
numeric_cars = cars[cols_continuous]

#delete rows with price with no data
numeric_cars.replace('?',np.nan, inplace=True)

#transform data to float
numeric_cars = numeric_cars.astype('float')

#numeric_cars.info()
#numeric_cars.head(10)

#delete row price null
numeric_cars.dropna(subset=['price'], inplace=True)

#count number of rows which are null
t = numeric_cars.isnull().sum()
print(t)

#replace nan with mean value
numeric_cars = numeric_cars.fillna(numeric_cars.mean())


#count number of rows which are null
t = numeric_cars.isnull().info()
print(t)

#normalize date between 0 and 1
col_price = numeric_cars["price"]
numeric_cars = ( numeric_cars - numeric_cars.min() ) / ( numeric_cars.max() - numeric_cars.min() )
numeric_cars["price"] = col_price

numeric_cars.head(5)

tot_rows = len(numeric_cars.index)
print(tot_rows)
#list of colums
#targetcols = cars.select_dtypes(include=['float']).columns.to_list()
#print(targetcols)

normalized-losses    36
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-size           0
bore                  4
stroke                4
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
price                 0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 203
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   normalized-losses  200 non-null    bool 
 1   wheel-base         200 non-null    bool 
 2   length             200 non-null    bool 
 3   width              200 non-null    bool 
 4   height             200 non-null    bool 
 5   curb-weight        200 non-null    bool 
 6   engine-size        200 non-null    bool 
 7   bore               200 non-null    bool 
 8   stroke             200 non-null    bool 
 9   compression

/Users/arnaudbreton/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


200


In [3]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

def knn_train_test(train_col, target_col, df):
    model = KNeighborsRegressor()
    np.random.seed(0)
    
    #shuffle data of df
    shuffled_index = np.random.permutation(df.index)
    rand_df = df.reindex(shuffled_index)
        
    #divide data in two
    last_train_row = int(len(rand_df / 2))
    
    #create train and test df
    train_df  = rand_df.iloc[0:last_train_row]
    test_df = rand_df.iloc[last_train_row:]
    
    #adjust knn model
    model.fit(train_df[[train_col]], train_df[target_col])
    predicted_labels = model.predict(test_df[[train_col]])
    
    #RMSE 
    mse = mean_squared_error(test_df[target_col],predicted_labels)
    rmse = np.sqrt(mse)
    
    return rmse


In [4]:
#dictionary
rmse_results = {}
train_cols = numeric_cars.columns.drop("price")

#we train the model
for col in train_cols:
    rmse_val = knn_train_test(col,"price",numeric_cars)
    rmse_results[col] = rmse_val
    
#create series of rmse results
rmse_result_series = pd.Series(rmse_results)
rmse_result_series.sort_values()
    

ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required.